In [ ]:
import os
import time
from matplotlib import pyplot as plt
from IPython.display import clear_output
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# !pip install git+https://github.com/tensorflow/examples.git

In [ ]:
tf.random.set_seed(789)

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

def load_dataset(path):
    dataset = tf.data.Dataset.list_files(path + '/*.*', shuffle=False)
    dataset = dataset.map(tf.io.read_file)
    dataset = dataset.map(tf.image.decode_jpeg, num_parallel_calls=AUTOTUNE)
    return dataset

# Load the training and test datasets
treinamento_A = load_dataset('CycleGAN/train/a')
treinamento_B = load_dataset('CycleGAN/train/b')
teste_A = load_dataset('CycleGAN/test/a')
teste_B = load_dataset('CycleGAN/test/b')


In [ ]:
quantidade_treinamento = tf.data.experimental.cardinality(treinamento_A)

# Preparação dos dados

In [ ]:
buffer_size = quantidade_treinamento
batch_size = 1
img_altura = 286
img_largura = 286

In [ ]:
def crop_aleratorio(imagem):
  imagem = tf.image.random_crop(imagem, size=[img_altura, img_largura, 3])
  return imagem

In [ ]:
def normalizar(imagem):
  imagem = tf.cast(imagem, tf.float32)
  imagem = (imagem / 127.5) - 1
  return imagem

In [ ]:
def jitter_aleatorio(imagem):
  imagem = tf.image.resize(imagem, [img_altura, img_largura], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
  imagem = crop_aleratorio(imagem)
  imagem = tf.image.random_flip_left_right(imagem)
  return imagem

In [ ]:
def processar_imagem_treinamento(imagem):
  imagem = jitter_aleatorio(imagem)
  imagem = normalizar(imagem)
  return imagem

In [ ]:
def processar_imagem_teste(imagem):
  imagem = normalizar(imagem)
  return imagem

In [ ]:
treinamento_A = treinamento_A.cache().map(processar_imagem_treinamento, num_parallel_calls=tf.data.experimental.AUTOTUNE).shuffle(buffer_size).batch(batch_size)
treinamento_B = treinamento_B.cache().map(processar_imagem_treinamento, num_parallel_calls=tf.data.experimental.AUTOTUNE).shuffle(buffer_size).batch(batch_size)
teste_A = teste_A.cache().map(processar_imagem_teste, num_parallel_calls=tf.data.experimental.AUTOTUNE).shuffle(buffer_size).batch(batch_size)
teste_B = teste_B.cache().map(processar_imagem_teste, num_parallel_calls=tf.data.experimental.AUTOTUNE).shuffle(buffer_size).batch(batch_size)

In [ ]:
amostra_A = next(iter(treinamento_A))
amostra_B = next(iter(treinamento_B))

In [ ]:
print(amostra_A.shape)
plt.figure(figsize=(6, 6))
for i in range(4):
  a_jitter = jitter_aleatorio(amostra_A[0])
  plt.subplot(2, 2, i+1)
  plt.imshow(a_jitter * 0.5 + 0.5)
  plt.axis('off')
plt.show()

In [ ]:
print(amostra_B.shape)
plt.figure(figsize=(6, 6))
for i in range(4):
  a_jitter = jitter_aleatorio(amostra_B[0])
  plt.subplot(2, 2, i+1)
  plt.imshow(a_jitter * 0.5 + 0.5)
  plt.axis('off')
plt.show()